In [1]:
import pandas as pd
import numpy as np

import time
import sys
from tqdm.notebook import tqdm

import torch
from transformers import CamembertModel, CamembertTokenizer

pd.set_option('display.max_colwidth', -1)

from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
path_data = "/Users/yohannlefaou/Documents/data/posos/"

In [3]:
train = pd.read_csv(path_data +"input_train.csv", sep=",")


In [4]:
labels = pd.read_csv(path_data + "output_train.csv", sep=",")

In [5]:
train.head()

,ID,question
0,0,"bonjour, je m suis trompé de forum pour ma question alors je la repose ici. je pris pour la première fois hier du paroxétine et ce matin c'est une catastrophe. picotement dasn tous le corps annonciateur de sueur froide très très massive et de vomissement. j'en suis à deux crises depuis 5 heure du mat. la cela semble passer mes mes mains reste moites et chaude estce normal pour la première fois merci a tous"
1,1,est ce que le motilium me soulagera contre les nausées?
2,2,"mon médecin m'a prescrit adenyl. au 2ème cachet des maux de tête terribles et au 3ème palpitations, sueurs froides, chaleur intense dans la tête, tremblements, fourmillements dans la lèvre supérieure, difficultés à respirer.. dès l'arrêt du médicament tous les symptômes ont disparu. cela est-il déjà arrivé à quelqu'un??"
3,3,Est-ce qu'il existe une forme adaptée aux enfant de 5ans du Micropakine ?
4,4,mon medecin me soigne pour une rhino pharingite et m'a prescrit du amoxicilline comme anti biotique. Est-ce vraiment pour cette indication?


In [6]:
labels.head()

,ID,intention
0,0,28
1,1,31
2,2,28
3,3,44
4,4,31


In [7]:
# load tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# load model
model = CamembertModel.from_pretrained('camembert-base')

In [10]:
text = train["question"][0]

# encode() automatically adds the classification token <s>
token_ids = tokenizer.encode(text)
tokens = [tokenizer._convert_id_to_token(idx) for idx in token_ids]
print(tokens)

# unsqueeze token_ids because batch_size=1
token_ids = torch.tensor(token_ids).unsqueeze(0)
print(token_ids)

# forward method returns a tuple (we only want the logits)
# squeeze() because batch_size=1
output = model(token_ids)[0].squeeze()
# only grab output of CLS token (<s>), which is the first token
cls_out = output[0]
print(cls_out.size())

['<s>', '▁bonjour', ',', '▁je', '▁m', '▁suis', '▁trompé', '▁de', '▁forum', '▁pour', '▁ma', '▁question', '▁alors', '▁je', '▁la', '▁repose', '▁ici', '.', '▁je', '▁pris', '▁pour', '▁la', '▁première', '▁fois', '▁hier', '▁du', '▁par', 'ox', 'é', 'tine', '▁et', '▁ce', '▁matin', '▁c', "'", 'est', '▁une', '▁catastrophe', '.', '▁pic', 'ote', 'ment', '▁d', 'as', 'n', '▁tous', '▁le', '▁corps', '▁', 'annon', 'ci', 'ateur', '▁de', '▁sueur', '▁froide', '▁très', '▁très', '▁massive', '▁et', '▁de', '▁vomi', 'ssement', '.', '▁j', "'", 'en', '▁suis', '▁à', '▁deux', '▁crises', '▁depuis', '▁5', '▁heure', '▁du', '▁mat', '.', '▁la', '▁cela', '▁semble', '▁passer', '▁mes', '▁mes', '▁mains', '▁reste', '▁moi', 'tes', '▁et', '▁chaude', '▁est', 'ce', '▁normal', '▁pour', '▁la', '▁première', '▁fois', '▁merci', '▁a', '▁tous', '</s>']
tensor([[    5,  5061,     7,    50,   115,   146, 12125,     8,  1026,    24,
           155,   397,   183,    50,    13,  4537,   323,     9,    50,   523,
            24,    13,   272

In [23]:
token_ids[:, :]

tensor([[    5,    50,  4608,  8331,   850,   451,    17,   286,   176,    20,
           318,     7,     8,   124,    15,   135, 26347,    10,    15,   135,
         19365,     7,   395,   129,   956,     8,   742,  5472,    15,   742,
          5228,    43,   118, 26347,    10,    15,  5378,     7,    27,    76,
            11,    73,  3470,    15,   124,    38,  2156, 10831,    10, 13770,
           280,   101,   120,    22,   732,   136,    49,    11, 12029,    34,
            72,    38, 12699,    10, 14443,    35,     7,   792,     8,   254,
             7,  2959,   142,    76,    11,    73,   380,    23,   281, 12960,
            18,    10,    23,    60,  3589,    31,    33,    67,  6834,    15,
           118, 26347,    10,    38, 12067,  2802,  5378,    53,   387, 14742,
           118, 26484,    10, 10263,    67,  6834,    15,   135, 26347,    10,
          3120,  2195,  5378,    53,   387, 14742,   135,  1075,   129,   956,
          1432,    95,  2935,    65,    34,   334,  

In [24]:
# compute embeddings for all questions
max_tokens = 512 # maximum allowed number of tokens in one sentence in order to compute embeddings
sentence_tokens = []
sentence_embeddings = []
for i in tqdm(range(len(train))):
    text = train["question"][i]
    token_ids = tokenizer.encode(text)
    token_ids = torch.tensor(token_ids).unsqueeze(0)
    sentence_tokens.append(token_ids)
    output = model(token_ids[:, :max_tokens])[0].squeeze()
    sentence_embeddings.append(output[0].tolist())

Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (966 > 512). Running this sequence through the model will result in indexing errors


In [25]:
train2 = pd.DataFrame(sentence_embeddings)

In [26]:
train2.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.103260,0.164496,0.000466,0.035828,-0.014292,-0.012606,-0.057755,0.201483,0.020815,0.107380,...,-0.003843,-0.086718,-0.041593,0.036421,0.165885,0.057104,-0.093255,-0.149246,-0.050350,0.122724
1,-0.085814,0.019714,0.092567,-0.046294,-0.063478,-0.012849,0.079008,0.265424,0.041190,0.084285,...,-0.116445,-0.077676,-0.152492,0.058571,0.083522,0.096404,0.073179,-0.092170,-0.057123,-0.005349
2,-0.075152,0.073966,0.039749,-0.000610,-0.024459,0.001018,-0.033008,0.212762,0.004864,0.096275,...,0.044050,-0.101421,-0.123798,0.122955,0.101871,0.067030,-0.021039,-0.177599,-0.051054,0.047357
3,0.032002,0.120783,0.066957,-0.128963,-0.008886,-0.034391,-0.008858,0.255312,0.078952,0.094861,...,-0.059976,0.025396,-0.195635,0.099526,0.057682,0.064332,0.017317,-0.061286,0.009324,-0.039828
4,-0.092328,0.184989,0.024131,-0.076040,0.028363,-0.058682,-0.020498,0.225975,0.007710,0.071678,...,-0.023249,0.029947,-0.145327,0.079080,0.088762,0.089566,0.032271,-0.190924,-0.045433,-0.005441


In [27]:
sys.getsizeof(train2)

49324192

In [23]:
train2.to_csv(path_data + "sentence_embeddings_CLS.csv", index=False)

In [8]:
train2 = pd.read_csv(path_data + "sentence_embeddings_CLS.csv")

# Neural network model

In [24]:
# preprocessing
# one hot encode the target
ohe = OneHotEncoder()
one_hot_encode_labels = ohe.fit_transform(labels["intention"].values.reshape(-1, 1)).toarray()

/Users/yohannlefaou/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(train2, one_hot_encode_labels, test_size=0.2,
                                                      random_state=2019)

In [26]:
nn = Sequential()
nn.add(Dropout(0.3, input_shape=(768,)))
#nn.add(Dense(1024, input_dim= 768, activation="relu")) # input_dim= 768
#nn.add(Dropout(0.3))
#nn.add(Dense(256, activation="relu"))
#nn.add(Dropout(0.3))
nn.add(Dense(51, activation="softmax")) #a completer

nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [27]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

nn.fit(np.array(X_train), y_train, epochs=30, batch_size=32,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 2s 384us/step - loss: 3.1423 - accuracy: 0.2154 - val_loss: 2.9775 - val_accuracy: 0.2335
Epoch 2/30
6422/6422 [==============================] - 2s 255us/step - loss: 2.8969 - accuracy: 0.2488 - val_loss: 2.8241 - val_accuracy: 0.2833
Epoch 3/30
6422/6422 [==============================] - 2s 266us/step - loss: 2.7528 - accuracy: 0.2867 - val_loss: 2.7028 - val_accuracy: 0.3088
Epoch 4/30
6422/6422 [==============================] - 2s 275us/step - loss: 2.6352 - accuracy: 0.3198 - val_loss: 2.6036 - val_accuracy: 0.3213
Epoch 5/30
6422/6422 [==============================] - 2s 269us/step - loss: 2.5355 - accuracy: 0.3440 - val_loss: 2.5197 - val_accuracy: 0.3624
Epoch 6/30
6422/6422 [==============================] - 2s 281us/step - loss: 2.4487 - accuracy: 0.3634 - val_loss: 2.4440 - val_accuracy: 0.3910
Epoch 7/30
6422/6422 [==============================] - 2s 277us/step - loss

In [28]:
nn.fit(np.array(X_train), y_train, epochs=10, batch_size=32,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/10
6422/6422 [==============================] - 2s 270us/step - loss: 1.6345 - accuracy: 0.5623 - val_loss: 1.7043 - val_accuracy: 0.5660
Epoch 2/10
6422/6422 [==============================] - 2s 259us/step - loss: 1.6203 - accuracy: 0.5646 - val_loss: 1.6972 - val_accuracy: 0.5573
Epoch 3/10
6422/6422 [==============================] - 2s 273us/step - loss: 1.5995 - accuracy: 0.5676 - val_loss: 1.6831 - val_accuracy: 0.5573
Epoch 4/10
6422/6422 [==============================] - 2s 281us/step - loss: 1.5910 - accuracy: 0.5744 - val_loss: 1.6700 - val_accuracy: 0.5660
Epoch 5/10
6422/6422 [==============================] - 2s 278us/step - loss: 1.5849 - accuracy: 0.5740 - val_loss: 1.6565 - val_accuracy: 0.5729
Epoch 6/10
6422/6422 [==============================] - 2s 293us/step - loss: 1.5652 - accuracy: 0.5847 - val_loss: 1.6500 - val_accuracy: 0.5654
Epoch 7/10
6422/6422 [==============================] - 2s 298us/step - loss

In [29]:
nn.fit(np.array(X_train), y_train, epochs=10, batch_size=32,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/10
6422/6422 [==============================] - 2s 269us/step - loss: 1.5098 - accuracy: 0.5944 - val_loss: 1.6079 - val_accuracy: 0.5803
Epoch 2/10
6422/6422 [==============================] - 2s 259us/step - loss: 1.5025 - accuracy: 0.5947 - val_loss: 1.5974 - val_accuracy: 0.5897
Epoch 3/10
6422/6422 [==============================] - 2s 276us/step - loss: 1.4971 - accuracy: 0.5990 - val_loss: 1.5892 - val_accuracy: 0.5890
Epoch 4/10
6422/6422 [==============================] - 2s 272us/step - loss: 1.4807 - accuracy: 0.5970 - val_loss: 1.5774 - val_accuracy: 0.5872
Epoch 5/10
6422/6422 [==============================] - 2s 274us/step - loss: 1.4776 - accuracy: 0.5987 - val_loss: 1.5765 - val_accuracy: 0.5897
Epoch 6/10
6422/6422 [==============================] - 2s 286us/step - loss: 1.4690 - accuracy: 0.5969 - val_loss: 1.5629 - val_accuracy: 0.5965
Epoch 7/10
6422/6422 [==============================] - 2s 283us/step - loss

In [30]:
nn.fit(np.array(X_train), y_train, epochs=30, batch_size=64,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 1s 184us/step - loss: 1.4204 - accuracy: 0.6127 - val_loss: 1.5314 - val_accuracy: 0.5978
Epoch 2/30
6422/6422 [==============================] - 1s 176us/step - loss: 1.4233 - accuracy: 0.6106 - val_loss: 1.5261 - val_accuracy: 0.6021
Epoch 3/30
6422/6422 [==============================] - 1s 192us/step - loss: 1.4153 - accuracy: 0.6132 - val_loss: 1.5257 - val_accuracy: 0.6009
Epoch 4/30
6422/6422 [==============================] - 1s 188us/step - loss: 1.3998 - accuracy: 0.6159 - val_loss: 1.5252 - val_accuracy: 0.6059
Epoch 5/30
6422/6422 [==============================] - 1s 189us/step - loss: 1.4095 - accuracy: 0.6159 - val_loss: 1.5199 - val_accuracy: 0.6015
Epoch 6/30
6422/6422 [==============================] - 1s 199us/step - loss: 1.4098 - accuracy: 0.6177 - val_loss: 1.5148 - val_accuracy: 0.6021
Epoch 7/30
6422/6422 [==============================] - 1s 191us/step - loss

KeyboardInterrupt: 

In [31]:
nn.fit(np.array(X_train), y_train, epochs=20, batch_size=128,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/5
6422/6422 [==============================] - 1s 136us/step - loss: 1.3696 - accuracy: 0.6236 - val_loss: 1.4747 - val_accuracy: 0.6108
Epoch 2/5
6422/6422 [==============================] - 1s 142us/step - loss: 1.3528 - accuracy: 0.6252 - val_loss: 1.4732 - val_accuracy: 0.6133
Epoch 3/5
6422/6422 [==============================] - 1s 142us/step - loss: 1.3469 - accuracy: 0.6291 - val_loss: 1.4717 - val_accuracy: 0.6152
Epoch 4/5
6422/6422 [==============================] - 1s 144us/step - loss: 1.3504 - accuracy: 0.6207 - val_loss: 1.4699 - val_accuracy: 0.6146
Epoch 5/5
6422/6422 [==============================] - 1s 146us/step - loss: 1.3510 - accuracy: 0.6314 - val_loss: 1.4677 - val_accuracy: 0.6158


In [32]:
nn.fit(np.array(X_train), y_train, epochs=20, batch_size=128,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/20
6422/6422 [==============================] - 1s 134us/step - loss: 1.3468 - accuracy: 0.6277 - val_loss: 1.4690 - val_accuracy: 0.6171
Epoch 2/20
6422/6422 [==============================] - 1s 144us/step - loss: 1.3401 - accuracy: 0.6322 - val_loss: 1.4685 - val_accuracy: 0.6146
Epoch 3/20
6422/6422 [==============================] - 1s 155us/step - loss: 1.3396 - accuracy: 0.6294 - val_loss: 1.4653 - val_accuracy: 0.6133
Epoch 4/20
6422/6422 [==============================] - 1s 142us/step - loss: 1.3395 - accuracy: 0.6341 - val_loss: 1.4639 - val_accuracy: 0.6133
Epoch 5/20
6422/6422 [==============================] - 1s 148us/step - loss: 1.3429 - accuracy: 0.6289 - val_loss: 1.4623 - val_accuracy: 0.6202
Epoch 6/20
6422/6422 [==============================] - 1s 151us/step - loss: 1.3393 - accuracy: 0.6274 - val_loss: 1.4616 - val_accuracy: 0.6121
Epoch 7/20
6422/6422 [==============================] - 1s 148us/step - loss

In [33]:
nn = Sequential()
nn.add(Dropout(0.1, input_shape=(768,)))
#nn.add(Dense(1024, input_dim= 768, activation="relu")) # input_dim= 768
#nn.add(Dropout(0.3))
#nn.add(Dense(256, activation="relu"))
#nn.add(Dropout(0.3))
nn.add(Dense(51, activation="softmax")) #a completer

nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

nn.fit(np.array(X_train), y_train, epochs=30, batch_size=64,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 2s 328us/step - loss: 3.2134 - accuracy: 0.1950 - val_loss: 3.0310 - val_accuracy: 0.2316
Epoch 2/30
6422/6422 [==============================] - 1s 192us/step - loss: 2.9575 - accuracy: 0.2270 - val_loss: 2.9050 - val_accuracy: 0.2522
Epoch 3/30
6422/6422 [==============================] - 1s 190us/step - loss: 2.8367 - accuracy: 0.2565 - val_loss: 2.8033 - val_accuracy: 0.2534
Epoch 4/30
6422/6422 [==============================] - 1s 188us/step - loss: 2.7333 - accuracy: 0.2846 - val_loss: 2.7175 - val_accuracy: 0.3001
Epoch 5/30
6422/6422 [==============================] - 1s 192us/step - loss: 2.6436 - accuracy: 0.3078 - val_loss: 2.6314 - val_accuracy: 0.3182
Epoch 6/30
6422/6422 [==============================] - 1s 210us/step - loss: 2.5600 - accuracy: 0.3306 - val_loss: 2.5628 - val_accuracy: 0.3618
Epoch 7/30
6422/6422 [==============================] - 1s 199us/step - loss

In [35]:
nn.fit(np.array(X_train), y_train, epochs=30, batch_size=64,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 1s 194us/step - loss: 1.6768 - accuracy: 0.5587 - val_loss: 1.7940 - val_accuracy: 0.5293
Epoch 2/30
6422/6422 [==============================] - 1s 194us/step - loss: 1.6556 - accuracy: 0.5631 - val_loss: 1.7844 - val_accuracy: 0.5380
Epoch 3/30
6422/6422 [==============================] - 1s 194us/step - loss: 1.6419 - accuracy: 0.5682 - val_loss: 1.7633 - val_accuracy: 0.5417
Epoch 4/30
6422/6422 [==============================] - 1s 197us/step - loss: 1.6233 - accuracy: 0.5741 - val_loss: 1.7560 - val_accuracy: 0.5492
Epoch 5/30
6422/6422 [==============================] - 1s 200us/step - loss: 1.6079 - accuracy: 0.5761 - val_loss: 1.7429 - val_accuracy: 0.5535
Epoch 6/30
6422/6422 [==============================] - 1s 192us/step - loss: 1.5967 - accuracy: 0.5846 - val_loss: 1.7264 - val_accuracy: 0.5486
Epoch 7/30
6422/6422 [==============================] - 1s 204us/step - loss

In [36]:
nn.fit(np.array(X_train), y_train, epochs=30, batch_size=64,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 1s 174us/step - loss: 1.3342 - accuracy: 0.6453 - val_loss: 1.5192 - val_accuracy: 0.5990
Epoch 2/30
6422/6422 [==============================] - 1s 178us/step - loss: 1.3257 - accuracy: 0.6450 - val_loss: 1.5109 - val_accuracy: 0.6077
Epoch 3/30
6422/6422 [==============================] - 1s 187us/step - loss: 1.3244 - accuracy: 0.6512 - val_loss: 1.5100 - val_accuracy: 0.6021
Epoch 4/30
6422/6422 [==============================] - 1s 181us/step - loss: 1.3088 - accuracy: 0.6500 - val_loss: 1.5040 - val_accuracy: 0.5984
Epoch 5/30
6422/6422 [==============================] - 1s 192us/step - loss: 1.3064 - accuracy: 0.6529 - val_loss: 1.5018 - val_accuracy: 0.6121
Epoch 6/30
6422/6422 [==============================] - 1s 181us/step - loss: 1.2997 - accuracy: 0.6501 - val_loss: 1.4917 - val_accuracy: 0.6108
Epoch 7/30
6422/6422 [==============================] - 1s 198us/step - loss

In [37]:
nn.fit(np.array(X_train), y_train, epochs=30, batch_size=124,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 1s 126us/step - loss: 1.1607 - accuracy: 0.6898 - val_loss: 1.3960 - val_accuracy: 0.6283
Epoch 2/30
6422/6422 [==============================] - 1s 131us/step - loss: 1.1616 - accuracy: 0.6856 - val_loss: 1.3959 - val_accuracy: 0.6295
Epoch 3/30
6422/6422 [==============================] - 1s 136us/step - loss: 1.1552 - accuracy: 0.6823 - val_loss: 1.3941 - val_accuracy: 0.6308
Epoch 4/30
6422/6422 [==============================] - 1s 147us/step - loss: 1.1548 - accuracy: 0.6867 - val_loss: 1.3900 - val_accuracy: 0.6308
Epoch 5/30
6422/6422 [==============================] - 1s 147us/step - loss: 1.1538 - accuracy: 0.6839 - val_loss: 1.3905 - val_accuracy: 0.6333
Epoch 6/30
6422/6422 [==============================] - 1s 185us/step - loss: 1.1527 - accuracy: 0.6819 - val_loss: 1.3894 - val_accuracy: 0.6295
Epoch 7/30
6422/6422 [==============================] - 1s 179us/step - loss

In [38]:
nn.fit(np.array(X_train), y_train, epochs=30, batch_size=256,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 1s 98us/step - loss: 1.0944 - accuracy: 0.6993 - val_loss: 1.3555 - val_accuracy: 0.6364
Epoch 2/30
6422/6422 [==============================] - 1s 109us/step - loss: 1.0851 - accuracy: 0.6990 - val_loss: 1.3562 - val_accuracy: 0.6413
Epoch 3/30
6422/6422 [==============================] - 1s 108us/step - loss: 1.0895 - accuracy: 0.7065 - val_loss: 1.3547 - val_accuracy: 0.6376
Epoch 4/30
6422/6422 [==============================] - 1s 121us/step - loss: 1.0922 - accuracy: 0.7007 - val_loss: 1.3535 - val_accuracy: 0.6357
Epoch 5/30
6422/6422 [==============================] - 1s 119us/step - loss: 1.0834 - accuracy: 0.6998 - val_loss: 1.3529 - val_accuracy: 0.6389
Epoch 6/30
6422/6422 [==============================] - 1s 114us/step - loss: 1.0900 - accuracy: 0.7009 - val_loss: 1.3526 - val_accuracy: 0.6413
Epoch 7/30
6422/6422 [==============================] - 1s 112us/step - loss:

In [39]:
nn = Sequential()
#nn.add(Dropout(0.3, input_shape=(768,)))
#nn.add(Dense(1024, input_dim= 768, activation="relu")) # input_dim= 768
#nn.add(Dropout(0.3))
#nn.add(Dense(256, activation="relu"))
#nn.add(Dropout(0.3))
nn.add(Dense(51, activation="softmax")) #a completer

nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

nn.fit(np.array(X_train), y_train, epochs=30, batch_size=32,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 2s 307us/step - loss: 3.1275 - accuracy: 0.2024 - val_loss: 2.9550 - val_accuracy: 0.2316
Epoch 2/30
6422/6422 [==============================] - 1s 195us/step - loss: 2.8554 - accuracy: 0.2560 - val_loss: 2.7833 - val_accuracy: 0.2740
Epoch 3/30
6422/6422 [==============================] - 1s 202us/step - loss: 2.6956 - accuracy: 0.2969 - val_loss: 2.6617 - val_accuracy: 0.3381
Epoch 4/30
6422/6422 [==============================] - 1s 209us/step - loss: 2.5654 - accuracy: 0.3335 - val_loss: 2.5528 - val_accuracy: 0.3611
Epoch 5/30
6422/6422 [==============================] - 1s 204us/step - loss: 2.4514 - accuracy: 0.3666 - val_loss: 2.4508 - val_accuracy: 0.3867
Epoch 6/30
6422/6422 [==============================] - 1s 211us/step - loss: 2.3570 - accuracy: 0.3927 - val_loss: 2.3725 - val_accuracy: 0.4016
Epoch 7/30
6422/6422 [==============================] - 1s 205us/step - loss

In [41]:
nn.fit(np.array(X_train), y_train, epochs=30, batch_size=64,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/30
6422/6422 [==============================] - 1s 116us/step - loss: 1.4203 - accuracy: 0.6310 - val_loss: 1.6071 - val_accuracy: 0.5841
Epoch 2/30
6422/6422 [==============================] - 1s 118us/step - loss: 1.4091 - accuracy: 0.6319 - val_loss: 1.6030 - val_accuracy: 0.5922
Epoch 3/30
6422/6422 [==============================] - 1s 120us/step - loss: 1.4009 - accuracy: 0.6313 - val_loss: 1.5972 - val_accuracy: 0.5872
Epoch 4/30
6422/6422 [==============================] - 1s 126us/step - loss: 1.3935 - accuracy: 0.6344 - val_loss: 1.5853 - val_accuracy: 0.5922
Epoch 5/30
6422/6422 [==============================] - 1s 131us/step - loss: 1.3857 - accuracy: 0.6391 - val_loss: 1.5803 - val_accuracy: 0.5897
Epoch 6/30
6422/6422 [==============================] - 1s 134us/step - loss: 1.3772 - accuracy: 0.6391 - val_loss: 1.5778 - val_accuracy: 0.5903
Epoch 7/30
6422/6422 [==============================] - 1s 126us/step - loss

In [43]:
nn.fit(np.array(X_train), y_train, epochs=10, batch_size=2048,
       validation_data=(np.array(X_valid), y_valid), shuffle=True) #, callbacks=[es]

Train on 6422 samples, validate on 1606 samples
Epoch 1/10
6422/6422 [==============================] - 0s 30us/step - loss: 1.1622 - accuracy: 0.6926 - val_loss: 1.4294 - val_accuracy: 0.6214
Epoch 2/10
6422/6422 [==============================] - 0s 31us/step - loss: 1.1627 - accuracy: 0.6937 - val_loss: 1.4293 - val_accuracy: 0.6208
Epoch 3/10
6422/6422 [==============================] - 0s 33us/step - loss: 1.1630 - accuracy: 0.6931 - val_loss: 1.4292 - val_accuracy: 0.6189
Epoch 4/10
6422/6422 [==============================] - 0s 32us/step - loss: 1.1628 - accuracy: 0.6932 - val_loss: 1.4287 - val_accuracy: 0.6177
Epoch 5/10
6422/6422 [==============================] - 0s 29us/step - loss: 1.1621 - accuracy: 0.6922 - val_loss: 1.4281 - val_accuracy: 0.6183
Epoch 6/10
6422/6422 [==============================] - 0s 30us/step - loss: 1.1613 - accuracy: 0.6923 - val_loss: 1.4277 - val_accuracy: 0.6189
Epoch 7/10
6422/6422 [==============================] - 0s 33us/step - loss: 1.160

# Export data

In [95]:
export_X_train, export_X_test, export_y_train, export_y_test = train_test_split(train,
                                                                                labels["intention"].values,
                                                                                test_size=0.2, 
                                                                                random_state=2019)

export_X_train["intention"] = export_y_train
export_X_test["intention"] = export_y_test

/Users/yohannlefaou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yohannlefaou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [99]:
export_X_train.to_csv("data/train.tsv", sep="\t", index=False)
export_X_train.to_csv("data/test.tsv", sep="\t", index=False)

In [97]:
dev = pd.read_csv(path_data +"input_test_b1Yip6O.csv", sep=",")

In [98]:
dev.to_csv("data/dev.tsv", sep="\t", index=False)